## **A note on python logging**

### There are 5 standard logging levels:


1. DEBUG: Detailed information, typically of interest only when diagnosing problems.
2. INFO: Confirmation that things are working as expected.
3. WARNING: An indication that something unexpected happened, or indicative of some problem in the near future (e.g. ‘disk space low’). The software is still working as expected.
4. ERROR: Due to a more serious problem, the software has not been able to perform some function.
5. CRITICAL: A serious error, indicating that the program itself may be unable to continue running.

*By deafult python will log warning,error and critical*

The difference between setting a level for a logger and setting a level for a file handler is in how logging messages are filtered and processed before being written to a log file:

✅ Logger level acts as a first filter (determining which messages get processed at all).

✅ Handler level further filters messages before they are written to a log file.

✅ You can have multiple handlers with different levels (e.g., console logs all messages, file only logs warnings/errors).

***Important*** 

✅ Every time you run this script, it adds a new handler, leading to duplicate log entries. Solution: Before adding a handler, check if one already exists.

✅ If you make any changes to the handler like the level or filename, you need to set them explicitly.

[LogRecord attributes](https://docs.python.org/3/library/logging.html#logrecord-attributes)



### Basic single level logging

In [ ]:
import logging
import machine # creates machine.log but not log_file.log

# change the default configuration, set a custom format to set how the log messages look like
# setting level = debug includes all levels from debug to critical
# logging.basicConfig(filename = 'log_file.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
# name - Name of the logger used to log the call.

logger = logging.getLogger(__name__) # will be main if we are running the file directly, if as an import then the filename
logger.setLevel(logging.DEBUG) # setting the level

# Prevent duplicate handlers
if not logger.hasHandlers():
    # Create a file handler
    file_handler = logging.FileHandler('./generated_log_files/log_file.log')
    
    # Set log format
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    # file_handler.setLevel(logging.INFO) # can set level for file handler

    # Add handler to logger
    logger.addHandler(file_handler)

# Dynamically change file handler level if needed
for handler in logger.handlers:
    if isinstance(handler, logging.FileHandler):
        handler.setLevel(logging.INFO)  # Change level dynamically

    
# Example log entry
logger.info("Logging setup complete!")



In [ ]:
def add(x, y):
    """Add Function"""
    return x + y


def subtract(x, y):
    """Subtract Function"""
    return x - y


def multiply(x, y):
    """Multiply Function"""
    return x * y


def divide(x, y):
    """Divide Function"""
    try:
        result = x/y
    except ZeroDivisionError as e:
        logger.error('Tried to divide by zero', exc_info=True) # another way to include the traceback
        # logger.exception('To include traceback for trying to divide by zero')
    else:
        return result


In [7]:
num_1 = 20
num_2 = 10

In [8]:
add_result = add(num_1, num_2)
logger.debug('Add: {}+{} = {}'.format(num_1, num_2, add_result))
logger.info('Addition was successful')

sub_result = subtract(num_1, num_2)
logger.debug('Sub: {}-{} = {}'.format(num_1, num_2, sub_result))
logger.info('Subtraction was successful')

mul_result = multiply(num_1, num_2)
logger.debug('Mul: {}*{} = {}'.format(num_1, num_2, mul_result))
logger.info('Multiplication was successful')

div_result = divide(num_1, num_2)
logger.debug('Div: {}/{} = {}'.format(num_1, num_2, div_result))
logger.info('Division was successful')

In [10]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Read the log file as a list of lines
with open('log_file.log', 'r') as file:
    lines = [line.strip() for line in file]

# print(lines) # print a list of logs

# Create a DataFrame
df = pd.DataFrame(lines, columns=['Logs'])

# Display the first few rows
df.head()



,Logs
0,"2025-04-02 12:00:54,567 - __main__ - INFO - Logging setup complete!"
1,"2025-04-02 12:01:07,598 - __main__ - INFO - Addition was successful"
2,"2025-04-02 12:01:07,599 - __main__ - INFO - Subtraction was successful"
3,"2025-04-02 12:01:07,600 - __main__ - INFO - Multiplication was successful"
4,"2025-04-02 12:01:07,601 - __main__ - INFO - Division was successful"


In [13]:
log_file_data = [line.split(' - ') for line in lines] # split turns string to a list
log_file_data

[['2025-04-02 12:00:54,567', '__main__', 'INFO', 'Logging setup complete!'],
 ['2025-04-02 12:01:07,598', '__main__', 'INFO', 'Addition was successful'],
 ['2025-04-02 12:01:07,599', '__main__', 'INFO', 'Subtraction was successful'],
 ['2025-04-02 12:01:07,600',
  '__main__',
  'INFO',
  'Multiplication was successful'],
 ['2025-04-02 12:01:07,601', '__main__', 'INFO', 'Division was successful'],
 ['2025-04-02 12:01:29,034', '__main__', 'INFO', 'Logging setup complete!'],
 ['2025-04-02 12:01:35,564', '__main__', 'DEBUG', 'Add: 20+10 = 30'],
 ['2025-04-02 12:01:35,564', '__main__', 'INFO', 'Addition was successful'],
 ['2025-04-02 12:01:35,565', '__main__', 'DEBUG', 'Sub: 20-10 = 10'],
 ['2025-04-02 12:01:35,565', '__main__', 'INFO', 'Subtraction was successful'],
 ['2025-04-02 12:01:35,565', '__main__', 'DEBUG', 'Mul: 20*10 = 200'],
 ['2025-04-02 12:01:35,570',
  '__main__',
  'INFO',
  'Multiplication was successful'],
 ['2025-04-02 12:01:35,571', '__main__', 'DEBUG', 'Div: 20/10 = 2.

In [12]:
log_df = pd.DataFrame(log_file_data, columns = ['Timestamp', 'Logger', 'Level', 'Message'])
log_df['Timestamp'] = pd.to_datetime(log_df['Timestamp'], format="%Y-%m-%d %H:%M:%S,%f")
log_df['Date'] = log_df['Timestamp'].dt.date
log_df['Time'] = log_df['Timestamp'].dt.time
log_df

,Timestamp,Logger,Level,Message,Date,Time
0,2025-04-02 12:00:54.567,__main__,INFO,Logging setup complete!,2025-04-02,12:00:54.567000
1,2025-04-02 12:01:07.598,__main__,INFO,Addition was successful,2025-04-02,12:01:07.598000
2,2025-04-02 12:01:07.599,__main__,INFO,Subtraction was successful,2025-04-02,12:01:07.599000
3,2025-04-02 12:01:07.600,__main__,INFO,Multiplication was successful,2025-04-02,12:01:07.600000
4,2025-04-02 12:01:07.601,__main__,INFO,Division was successful,2025-04-02,12:01:07.601000
5,2025-04-02 12:01:29.034,__main__,INFO,Logging setup complete!,2025-04-02,12:01:29.034000
6,2025-04-02 12:01:35.564,__main__,DEBUG,Add: 20+10 = 30,2025-04-02,12:01:35.564000
7,2025-04-02 12:01:35.564,__main__,INFO,Addition was successful,2025-04-02,12:01:35.564000
8,2025-04-02 12:01:35.565,__main__,DEBUG,Sub: 20-10 = 10,2025-04-02,12:01:35.565000
9,2025-04-02 12:01:35.565,__main__,INFO,Subtraction was successful,2025-04-02,12:01:35.565000


In [14]:
log_df.to_csv('logfile.csv', index = False) # convert the log file content to a csv